In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-10-15 17:24:56.612654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 17:24:56.612696: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
runs = 5

ds_name = 'sst2'
test_dataset = None
eval_mapping = '[]'
num_labels = 2
valid_split = 'validation'
test_split = 'validation'

db_path = "./temp.db"
seed = 0
pool_size = 1000
search_size = 100
warmup_runs = 500
annealing_runs = 1000
wandb_project = 'sst_search_test'
wandb_entity = 'johntzwei'

model_card = "google/electra-small-discriminator"
pretraining = True
max_steps = 500
eval_steps = 500
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
model_card = "roberta-base"
db_path = "../../results/sst_genetic.db"
search_size = 100

In [5]:
training_args = select.SubsetTrainingArguments(num_labels=num_labels,
                                               model_card=model_card,
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(db_path=db_path,
                                                seed=seed, 
                                                data_pool_size=pool_size,
                                                optimal_subset_size=search_size,
                                                warmup_runs=warmup_runs,
                                                annealing_runs=annealing_runs,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [6]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-53804d3571838bad.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6257b67c9611994c.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-69a14aade0efa57f.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4191b237bd8001be.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-ae483633eb1f52ef.arrow
Loading cached processed dataset at /home/johnny/.cache

In [7]:
processed_ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'scalar_label', 'tokens', 'tree', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2210
    })
})

In [8]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds[valid_split], test_ds=processed_ds[test_split]
)

In [9]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(db_path)
df = df.sort_values('objective')
idx = json.loads(df.iloc[-1]['indexes'])

In [10]:
len(idx)

100

In [11]:
select_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    select_quality.append(quality)

wandb: Currently logged in as: johntzwei (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-10-15 17:25:07.165338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 17:25:07.165380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/500 [00:00<?, ?it/s]

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.2926983021899106 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.613371217412041 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.197133175595793 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.512195697044009 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 8.203246525757523 seconds), retrying request


  0%|          | 0/138 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/500 [00:00<?, ?it/s]

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.2832290772387487 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.182859355825458 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.755453323549264 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.211996711217652 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.889740999015758 seconds), retrying request


  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

In [12]:
select_quality

[0.8192552225249773,
 0.7883742052679382,
 0.8038147138964578,
 0.8083560399636693,
 0.8147138964577657]

In [13]:
np.mean(select_quality), np.std(select_quality)

(0.8069028156221618, 0.010663513258084784)

In [14]:
random_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    idx = np.random.choice(np.arange(pool_size), size=search_size)
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    random_quality.append(quality)
random_quality

loss,████▇█▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_test:accuracy,▁
sst:val_acc,▁
loss,0.00036
sst2_test:accuracy,0.81471
sst:val_acc,0.81471


wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-10-15 17:31:07.747369: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 17:31:07.747403: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/500 [00:00<?, ?it/s]

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.302377298037247 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.314969929079409 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.1221456499211055 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.709358447022806 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.014492928953373 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.883151130326365 seconds), retrying request


...

  0%|          | 0/500 [00:00<?, ?it/s]

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.284050826061301 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.659992342331755 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.080538663541554 seconds), retrying request


  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

[0.8056312443233424,
 0.812897366030881,
 0.8292461398728429,
 0.7693006357856494,
 0.8174386920980926]

In [15]:
random_quality

[0.8056312443233424,
 0.812897366030881,
 0.8292461398728429,
 0.7693006357856494,
 0.8174386920980926]

In [16]:
np.mean(random_quality), np.std(random_quality)

(0.8069028156221616, 0.020306177820137494)

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.193629747931658 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.4708724099220944 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.1147032495825067 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.1230148198919783 seconds), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.300202004850159 seconds), retrying request
